In [2]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
torch.cuda.empty_cache()

In [3]:
from transformers import T5Tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

def preprocess_and_tokenize(file_path):
    """
    Reads the file, preprocesses it, and tokenizes the data.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    input_texts = []
    target_labels = []
    
    current_tokens = []
    current_labels = []
    
    for line in lines:
        line = line.strip()
        if line:  # Non-empty line
            token, label = line.split()
            current_tokens.append(token)
            current_labels.append(label)
        else:  # Empty line signifies end of a sentence
            if current_tokens:
                input_texts.append(' '.join(current_tokens))
                target_labels.append(' '.join(current_labels))
                # Reset for next sentence
                current_tokens = []
                current_labels = []

    # Add the last sentence if the data doesn't end with an empty line
    if current_tokens:
        input_texts.append(' '.join(current_tokens))
        target_labels.append(' '.join(current_labels))
    
    return input_texts, target_labels


# Example usage
file_path = 'NER/Training_Data/AutoLabelledSet.txt'  # Replace with your file path
input_texts, target_labels = preprocess_and_tokenize(file_path)
print(input_texts[:5])
print(target_labels[:5])

['the jubilee domain contains a complex sequence of ultramafic mafic rocks and interleaved sedimentary rocks , overlain by polymictic conglomerate .', 'extensive weathered banded quartz magnetite rocks occur throughout the project area .', 'the most common and significant metaliferrous rock types in the area are metamorphosed banded iron formations , ( bif ) and granular iron formations , ( gif ) .', 'these can be either completely oxidised medium to coarse grain hematite or more stable magnetite .', 'a major volcanic centre , defined by abundant felsic volcanics and quartz aluminosilicatechloritoid rocks ( considered to represent metamorphosed alteration assemblages ) , occurs in the central portion of this domain .']
['O B-LOCATION O O O O O O B-ROCK I-ROCK I-ROCK O O B-ROCK I-ROCK O O O B-ROCK I-ROCK O', 'O O O B-MINERAL B-MINERAL O O O O O O O', 'O O O O O B-ROCK I-ROCK O O O O O O B-ROCK I-ROCK I-ROCK O O B-ROCK O O B-ROCK I-ROCK I-ROCK O O B-ROCK O O', 'O O O O O O O O O O B-MINE

In [4]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    input_texts, target_labels, test_size=0.2, random_state=42
)

print(f"Number of training samples: {len(train_texts)}")
print(f"Number of testing samples: {len(test_texts)}")

print(train_texts[:5])

Number of training samples: 25554
Number of testing samples: 6389
['4.2 local geology the project is underlain entirely by rocks of the coolgardie domain , a sub domain of the kalgoorlie terrane .', 'the abundant crystals or phenocrysts are totally altered feldspars and biotite plus the quartz .', 'recent work by blina resources nl has announced the discovery of numerous diamond bearing palaeochannels on their tenements held in joint venture with the kimberley diamond company immediately to the east of nwds kimberley downs prospects .', 'exploration comprised prospecting , rock sampling and stream sediment geochemistry .', 'historical drilling and surface sampling gold deposit legend historical drilling historical surface sampling cheroona well project 2006 annual report page 26 of 35 5.0 previous work gleneagle gold limited during the previous reporting period from 20th july 2005 to 31st march 2006 , exploration conducted by gleneagle gold limited included the processing of geophysica

In [5]:
# Initialize tokenizer
# tokenizer = T5Tokenizer.from_pretrained("t5-small")
tokenizer = AutoTokenizer.from_pretrained("dbmdz/t5-base-conll03-english")

def tokenize_data(texts, labels):
    """
    Tokenizes the input texts and labels.
    """
    inputs = tokenizer(texts, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
    targets = tokenizer(labels, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
    
    # Ensure labels are the same length as inputs
    inputs['labels'] = targets['input_ids']
    
    return inputs

# Tokenize training and testing data
train_data = tokenize_data(train_texts, train_labels)
test_data = tokenize_data(test_texts, test_labels)

# Print tokenized data for verification
print("Training Input IDs:", train_data['input_ids'])
print("Training Labels:", train_data['labels'])
print("Testing Input IDs:", test_data['input_ids'])
print("Testing Labels:", test_data['labels'])


Training Input IDs: tensor([[    3, 19765,   415,  ...,     0,     0,     0],
        [    8, 16346,  6884,  ...,     0,     0,     0],
        [ 1100,   161,    57,  ...,     0,     0,     0],
        ...,
        [   48,  3303,    65,  ...,     0,     0,     0],
        [    8, 11508,     9,  ...,     0,     0,     0],
        [ 2724,  4012,  2155,  ...,     0,     0,     0]])
Training Labels: tensor([[411, 411, 411,  ...,   0,   0,   0],
        [411, 411, 272,  ...,   0,   0,   0],
        [411, 411, 411,  ...,   0,   0,   0],
        ...,
        [411, 411, 411,  ...,   0,   0,   0],
        [411, 272,  18,  ...,   0,   0,   0],
        [411, 411, 411,  ...,   0,   0,   0]])
Testing Input IDs: tensor([[   12,     8,  3457,  ...,     0,     0,     0],
        [    3,     9,  4727,  ...,     0,     0,     0],
        [ 8282,  1467,    13,  ...,     0,     0,     0],
        ...,
        [    8,  3731,   159,  ...,     0,     0,     0],
        [ 1877,  2122,  2252,  ...,     0,     

In [6]:
import torch

class NERDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx].clone().detach()
        return item

    def __len__(self):
        return len(self.labels)


In [8]:
# Prepare datasets
train_dataset = NERDataset(train_data, train_data['labels'])
test_dataset = NERDataset(test_data, test_data['labels'])

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, num_workers=12, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=12)

In [17]:
from transformers import Trainer, TrainingArguments, T5ForConditionalGeneration

# Initialize model
model = AutoModelForSeq2SeqLM.from_pretrained("dbmdz/t5-base-conll03-english").to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="steps",
    gradient_accumulation_steps=8,
    save_steps=10000,
    eval_steps=10000,
    fp16=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
)


# Define metrics computation
from sklearn.metrics import f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Flatten the lists
    labels = labels.flatten()
    predictions = predictions.flatten()
    return {
        'f1': f1_score(labels, predictions, average='weighted')
    }

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)



c:\Users\Peter\miniconda3\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [18]:
# Train the model
# trainer.train()

  0%|          | 0/7980 [00:00<?, ?it/s]

{'loss': 0.168, 'grad_norm': 0.06683902442455292, 'learning_rate': 4.93734335839599e-05, 'epoch': 0.13}
{'loss': 0.0681, 'grad_norm': 0.058328814804553986, 'learning_rate': 4.8746867167919805e-05, 'epoch': 0.25}
{'loss': 0.0586, 'grad_norm': 0.10578988492488861, 'learning_rate': 4.81203007518797e-05, 'epoch': 0.38}
{'loss': 0.0527, 'grad_norm': 0.08469075709581375, 'learning_rate': 4.74937343358396e-05, 'epoch': 0.5}
{'loss': 0.0463, 'grad_norm': 0.2472144067287445, 'learning_rate': 4.6867167919799495e-05, 'epoch': 0.63}
{'loss': 0.0388, 'grad_norm': 0.29074814915657043, 'learning_rate': 4.62406015037594e-05, 'epoch': 0.75}
{'loss': 0.0361, 'grad_norm': 0.2826578915119171, 'learning_rate': 4.56140350877193e-05, 'epoch': 0.88}
{'loss': 0.0314, 'grad_norm': 0.16509296000003815, 'learning_rate': 4.49874686716792e-05, 'epoch': 1.0}
{'loss': 0.0283, 'grad_norm': 0.17966510355472565, 'learning_rate': 4.43609022556391e-05, 'epoch': 1.13}
{'loss': 0.0272, 'grad_norm': 0.18065638840198517, 'lea

TrainOutput(global_step=7980, training_loss=0.01668532095606763, metrics={'train_runtime': 21355.5069, 'train_samples_per_second': 11.966, 'train_steps_per_second': 0.374, 'train_loss': 0.01668532095606763, 'epoch': 9.99})

In [5]:
def tokenize_data(texts, tokenizer, max_length=256):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

def predict_entities(texts, model, tokenizer):
    inputs = tokenize_data(texts, tokenizer)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient calculation
        outputs = model.generate(**inputs, max_new_tokens=256)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)


# Example geological text
geo_texts = [
    "the jubilee domain contains a complex sequence of ultramafic mafic rocks and more evolved rocks, due to magma mixing and fractional crystallization .",
]

# Predict geological entities
predictions = predict_entities(geo_texts, model, tokenizer)

# Print the results
print(predictions)
for text, prediction in zip(geo_texts, predictions):
    print(f"Text: {text}")
    print(f"Prediction: {prediction}")
    print()

['O B-LOCATION O O O O O O B-ROCK I-ROCK I-ROCK O O O O O O O O O O O O O']
Text: the jubilee domain contains a complex sequence of ultramafic mafic rocks and more evolved rocks, due to magma mixing and fractional crystallization .
Prediction: O B-LOCATION O O O O O O B-ROCK I-ROCK I-ROCK O O O O O O O O O O O O O



In [23]:
# model.save_pretrained('./saved_model')  # Custom directory for saving
# tokenizer.save_pretrained('./saved_model')  # Save tokenizer as well

('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\tokenizer.json')

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

save_directory = "./NER/saved_model"
# Load the tokenizer and model from the saved directory
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory).to(device)